In [ ]:
from propeller import PropellerGeometry
from mesh import PropellerMesh
from dynamics import SixDOFDynamics
from static import QuadcopterForceMoments
from vlm import UVLM
from wind import WindField
from scipy.interpolate import UnivariateSpline
from scipy.io import savemat
from mpl_toolkits.mplot3d import Axes3D
from propeller import PropellerGeometry
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numba as nb
import copy
from matplotlib.gridspec import GridSpec
import pyvista as pv
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection


#-----------------------------------------------------------------------------------------------------------------------------------------------------------------_#
wake_states = []  # List to store states at each time step

# Initialize geometry and mesh
propeller_geometry = PropellerGeometry(
    airfoil_distribution_file='DJI9443_airfoils.csv',
    chorddist_file='DJI9443_chorddist.csv',
    pitchdist_file='DJI9443_pitchdist.csv',
    sweepdist_file='DJI9443_sweepdist.csv',
    heightdist_file='DJI9443_heightdist.csv',
    R_tip=0.11938,
    R_hub=0.00624,
    num_blades=2
)

propeller_mesh_system = PropellerMesh(propeller_geometry, arm_length=1.35, com=(0, 0, 0))
quad_propeller_mesh = propeller_mesh_system.generate_quad_propeller_mesh()

# Initialize SixDOF dynamics
mass = 0.68  # kg
inertia_matrix = np.array([
    [1.0, 0, 0],
    [0, 1.0, 0],
    [0, 0, 1.0]

])  # Simple diagonal inertia matrix for example

initial_position = np.array([0, 0, 0])  # Starting position
initial_body_velocity = np.array([0, 0, 0])  # Starting velocity
initial_orientation = np.array([0, 0, 0])  # Starting roll, pitch, yaw
initial_angular_velocity = np.array([0, 0, 0])  # Starting angular rates

omega_dict = {  # Motor speed initialization
    'Propeller_1': np.array([0, 0, 565.9]),
    'Propeller_2': np.array([0, 0, -565.9]),
    'Propeller_3': np.array([0, 0, -565.9]),
    'Propeller_4': np.array([0, 0, 565.9])
}


# Initialize the UVLM solver
uvlm_solver = UVLM(quad_propeller_mesh)
wake_states = []

dynamics = SixDOFDynamics(
    mass=mass,
    inertia_matrix=inertia_matrix,
    initial_position=np.zeros(3),
    initial_velocity_body=np.zeros(3),
    initial_angles=np.zeros(3),
    initial_angular_rates=np.zeros(3)
)

total_time = 5 
dt = 5  #
time_steps = int(total_time / dt)

wake_history = []
time_points = []
normalized_forces = []
rho = 1.071778   
rpm = 90
force_moment_calc = QuadcopterForceMoments(
    arm_length=0.5, 
    com_position=[0, 0, 0]  
)

mesh = pv.read(r"C:\Users\KU5001153\Downloads\bp_50percent_75.vtk") 
wind_field = WindField(mesh)

moment_history = {
    'com_x': [],     # Store COM x positions 
    'total_moments': []  # Store total moments array [mx, my, mz]
}

com_position = np.array([80, 125, 50])
n_revs = 2
n_steps_rev = 20
n_steps = n_revs * n_steps_rev
dt = 2*np.pi / (565.9 * n_steps_rev)

time_steps = 2

for time_step in range(1, 2):
    # Update COM position (x decreases by 1 each step until 200)
    # com_position[0] = (200 - time_step)

    # Step 1: Calculate forces and moments using UVLM
    forces_and_moments = uvlm_solver.calculate_total_forces_and_moments(
        propeller_mesh=quad_propeller_mesh,
        dt=dt,
        rho=rho,
        time_step=time_step,
        body_velocity=dynamics.velocity_body,  # Now using body-frame velocity
        omega=omega_dict,
        wind_field=wind_field,
        com_position=com_position,
        roll=dynamics.angles[0],
        pitch=dynamics.angles[1],
        yaw=dynamics.angles[2]
    )
    propeller_1_force = forces_and_moments['Propeller_1']['force']
    total_force = np.zeros(3)
    total_moment = np.zeros(3)

    for prop_force in propeller_1_force:
        total_force += propeller_1_force

    total_moment = np.zeros(3)
    for prop_key in ['Propeller_1', 'Propeller_2', 'Propeller_3', 'Propeller_4']:
        total_moment += forces_and_moments[prop_key]['moment']
   
    moment_history['com_x'].append(float(com_position[0]))
    moment_history['total_moments'].append(total_moment.tolist())
        
    # time_points.append(current_time)
    # normalized_forces.append(normalized_force)
    # dynamics.update_states(total_force, total_moment, dt, current_time)
    
    # Step 3: Update mesh geometry for both body motion and propeller rotation
    SixDOFDynamics.update_mesh_transformations(
        propeller_mesh=quad_propeller_mesh,
        dynamics=dynamics,
        dt=dt,
        omega_dict=omega_dict
    )

    # fixed_radial_index = 1 
    # uvlm_solver.plot_gamma_distribution(quad_propeller_mesh, fixed_radial_index)
    # uvlm_solver.plot_detailed_gamma_distribution(quad_propeller_mesh)
    # # uvlm_solver.plot_quad_propeller_pressure(
    # #     quad_propeller_mesh,
    # #     omega=omega_dict,  # Your angular velocity vector
    # #     body_velocity=dynamics.velocity_body,       # Your freestream velocity
    # #     rho=1.225
    # # uvlm_solver.plot_wake_system(quad_propeller_mesh, uvlm_solver.wake_system, "Propeller_1", show_bound=True)
        
    # # )

    uvlm_solver.plot_blades_pressure_top_view(
    quad_propeller_mesh,
    propeller_key="Propeller_1",
    omega=omega_dict["Propeller_1"], 
    body_velocity=initial_body_velocity,   
    rho=rho
)
    # propeller_geometry.plot_distributions()
    # print(f"Completed time step {time_step}")
    # print("-" * 50)

def plot_moments_separately(moment_history):
    """Create three separate plots for each moment component vs COM x-position with improved aesthetics."""
    fig = plt.figure(figsize=(12, 12))
    gs = GridSpec(3, 1, height_ratios=[1, 1, 1], hspace=0.3)
    
    # Your specified colors
    colors = ['#FB8861', '#F7D43B', '#9C179E']  # Red and Yellow (used twice as specified)
    
    # Extract data
    com_x = moment_history['com_x']
    moments = np.array(moment_history['total_moments'])
    
    # Create subplots
    axes = []
    titles = ['Moment X', 'Moment Y', 'Moment Z']
    
    for i in range(3):
        ax = fig.add_subplot(gs[i])
        axes.append(ax)
        
        # Plot data
        ax.plot(com_x, moments[:, i], color=colors[i], linewidth=2.5)
        
        # Customize appearance
        ax.set_xlabel('COM X Position (m)', fontsize=12)
        ax.set_ylabel(f'{titles[i]} (N⋅m)', fontsize=12)
        ax.set_title(f'{titles[i]} vs COM Position', fontsize=14, pad=15)
        
        # Customize grid
        ax.grid(True, linestyle='--', alpha=0.1)
        ax.tick_params(axis='both', labelsize=10)
        
        # Customize spines
        ax.spines['bottom'].set_color('black')
        ax.spines['top'].set_color('black')
        ax.spines['left'].set_color('black')
        ax.spines['right'].set_color('black')
        
        # Set background color
        ax.set_facecolor('white')

    # Set figure title and background
    fig.suptitle('Total Moments vs COM Position', fontsize=16, y=0.95)
    fig.patch.set_facecolor('white')
    
    plt.tight_layout()
    plt.show()
    
# After your simulation loop completes, call the plotting function:
plot_moments_separately(moment_history)

